In [3]:
from manim import *

from selma.datastructures import *

In [4]:
%%manim --renderer=opengl --hide-splash -pqm -v WARNING FirstScene

class FirstScene(Scene):
  def construct(self):
    
    Q = MQueue(scale=.5)
    Q.rect.to_edge(UP)
    self.add(Q.rect)
    
    def re(element):
      text = Text(element, color=WHITE)
      return VGroup(text, Rectangle(width = text.width + 0.4, height = .8, color=RED)).scale(.5)

    self.play(Q.enqueue(re("Hello")))
    self.play(Q.enqueue(re("world")))
    self.play(Q.dequeue())
    self.play(Q.enqueue(re("this")))
    self.play(Q.enqueue(re("is")))
    self.play(Q.dequeue())
    self.play(Q.dequeue())
    self.play(Q.dequeue())
    

KeyboardInterrupt: 

In [ ]:
%%manim --hide-splash -qm -v WARNING SecondScene

class SecondScene(Scene):
  def construct(self):
    
    S = MStack(scale=.5)
    S.rect.to_edge(UP)
    self.add(S.rect)
    
    def re(element):
      text = Text(element, color=WHITE)
      return VGroup(text, Rectangle(width = text.width + 0.4, height = .8, color=RED)).scale(.5)

    S.push(re("Hello")).play(self)    
    S.push(re("world")).play(self)
    S.pop().play(self)
    S.push(re("this")).play(self)
    S.push(re("is")).play(self)
    S.pop().play(self)
    S.pop().play(self)
    S.pop().play(self)
    